In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd
import scipy.sparse
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Circle
import seaborn as sns; 
from IPython.display import HTML

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import r2_score

In [ ]:
#config parser
import configparser

sys.path.insert(0, '../Common/')
from AISDataManager import AISDataManager
import Constants as c
import HMUtils as hMUtil
import TimeUtils as timeUtils
import GeoCompute as gC

#MyConfig.INI stores all the run time constants
config = configparser.ConfigParser()
config.read('../MyConfig.INI')

from joblib import Parallel, delayed
import multiprocessing
aISDM = AISDataManager()
numCores = multiprocessing.cpu_count()

In [ ]:
lonMin = (float)(config['TP_SEC_ORDER']['LON_MIN'])
lonMax = (float)(config['TP_SEC_ORDER']['LON_MAX'])

latMin = (float)(config['TP_SEC_ORDER']['LAT_MIN'])
latMax = (float)(config['TP_SEC_ORDER']['LAT_MAX'])

print(lonMin,latMin)
print(lonMax,latMax)

increStep = (float)(config['TP_SEC_ORDER']['INCR_STEP'])
incrRes = (int)(config['TP_SEC_ORDER']['INCR_RES'])

fileDir = config['TP_SEC_ORDER']['SOURCE_DIR']
dirToStore = config['TP_SEC_ORDER']['DEST_DIR']

print("SOURCE_DIR = %s"%fileDir)
print("DEST_DIR = %s"%dirToStore)

In [ ]:
heatMapGrid = hMUtil.generate_grid(lonMin, lonMax, latMin, latMax, increStep, incrRes)

In [ ]:
boundaryArray = heatMapGrid[2]
horizontalAxis = heatMapGrid[0]
verticalAxis = heatMapGrid[1]
totalStates = horizontalAxis.shape[0] * verticalAxis.shape[0]

In [ ]:
opFile = dirToStore + 'TPM.npz'
# opFile = dirToStore + 'TPM_.npz'
tpMatrix = scipy.sparse.load_npz(opFile)

# opFile = dirToStore + 'TPM.npy'
# tpMatrix = np.load(opFile)

In [ ]:
print(tpMatrix.sum())

In [ ]:
def get_vessel_init_status(lonStart,latStart):
    vesselState = np.zeros((horizontalAxis.shape[0]*verticalAxis.shape[0]))
    for boundary in boundaryArray :
        if((lonStart >= boundary[0]) \
            and (lonStart < boundary[1]) \
           
            and (latStart >= boundary[2]) \
            and (latStart < boundary[3]) \
          ):
            vesselState[boundary[4]] = 1.0
            break
    return vesselState

In [ ]:
#takes inital state vector
#and number of steps 
#uses transition probabilities to generate the path
def generate_path(vesselStatus):
    return (vesselStatus.T @ tpMatrix).flatten()

In [ ]:
def generate_vessel_prev_cur_status(prevState,currState):
    vesselState = np.zeros((totalStates*totalStates))
    stateCounter = 0
    for i in range(prevState.shape[0]):
        for j in range(currState.shape[0]):
            vesselState[stateCounter] = prevState[i]*currState[j]
            stateCounter = stateCounter + 1
    return vesselState

In [ ]:

# oneVessel = "../Data/M120_50_M119_00_33_90_34_38/MMSI/566952000_Sorted.csv"
# oneVessel = "../Data/M120_00_M119_50_34_12_34_24/MMSI/636012801_Sorted.csv"
# oneVessel = "../Data/M119_50_M119_00_34_00_34_16/MMSI/367353080_Sorted.csv"
# oneVessel = "../Data/M119_50_M119_00_34_00_34_16/MMSI/636012801_Sorted.csv"
#For fishing
# oneVessel = "../Data/M120_50_M119_00_33_90_34_44/MMSI/367481870_Sorted.csv"
# oneVessel = "../Data/M120_50_M119_00_33_90_34_44/MMSI/367457870_Sorted.csv"
# oneVessel = "../Data/M120_50_M119_00_33_50_33_90/MMSI/352135000_Sorted.csv"
# oneVessel = "../Data/M120_50_M119_00_33_90_34_44/MMSI/374128000_Sorted.csv"

# oneVessel = "../Data/M120_50_M119_00_33_90_34_44/MMSI/355649000_Sorted.csv"
oneVessel = "../Data/M120_50_M119_00_33_90_34_44/MMSI/565819000_Sorted.csv"
# oneVessel = "../Data/M120_50_M119_00_33_90_34_44/MMSI/636091434_Sorted.csv"
# oneVessel = "../Data/M119_00_M118_00_33_50_33_90/MMSI/355856000_Sorted.csv"
# oneVessel = "../Data/M119_00_M118_00_33_50_33_90/MMSI/477316000_Sorted.csv"

fix,ax = plt.subplots()
oneVesselData,_ = aISDM.load_data_from_csv(oneVessel)
print(oneVesselData.shape)
oneVesselLon = oneVesselData['LON']
oneVesselLat = oneVesselData['LAT']
ax.set_xlim(lonMin, lonMax)
ax.set_ylim(latMin, latMax)
ax.plot(oneVesselLon,oneVesselLat)

In [ ]:
oneVesselTime = oneVesselData['DateTime']
for i in range(oneVesselLon.shape[0]):
    print(i,oneVesselTime[i])

In [ ]:
timeIntervalList = "../Data/TimeInterval/HalfHourIntvl1701To1702_00.txt"
# timeIntervalList = "../Data/TimeInterval/HalfHourIntvl1702To1703_00.txt"
# timeIntervalList = "../Data/TimeInterval/HalfHourIntvl1704To1705_00.txt"
# timeIntervalList = "../Data/TimeInterval/HalfHourIntvl1706To1707_00.txt"
# timeIntervalList = "../Data/TimeInterval/HalfHourIntvl1704To1704_00.txt"
# timeIntervalList = "../Data/TimeInterval/HalfHourIntvl1711To1712_00.txt"
# timeIntervalList = "../Data/TimeInterval/HalfHourIntvl1712To1801_00.txt"
# timeIntervalList = "../Data/TimeInterval/HalfHourIntvl1712To1801_00.txt"
timeWindows = [line.rstrip('\n') for line in open(timeIntervalList)]

In [ ]:
#get the last instance of the position data 
#for one of the vessel
#and that can be used for the prediction
oneVesselLastData = pd.DataFrame()
#based on time stamps 
for timeSlot in timeWindows:
    temp = timeSlot.split(',')
    startTime = temp[0]
    endTime = temp[1]

    timelyDF = aISDM.filter_based_on_time_stamp(oneVesselData,'DateTime',startTime,endTime)
    oneVesselRows = timelyDF.shape[0]
    
    invertedTimelyDF = aISDM.inver_df(timelyDF)
    invertedTimelyDF = invertedTimelyDF.drop_duplicates(subset="MMSI")
    
    oneVesselLastData = oneVesselLastData.append(invertedTimelyDF, ignore_index = True)

In [ ]:
oneVesselTime = oneVesselLastData['DateTime']
# oneVesselTime = oneVesselLastData['LON']
for i in range(oneVesselTime.shape[0]):
    print(i,oneVesselTime[i])

In [ ]:
oneVesselLon = oneVesselLastData['LON']
oneVesselLat = oneVesselLastData['LAT']
for i in range(oneVesselLon.shape[0]):
    print(i,oneVesselLon[i],oneVesselLat[i])

In [ ]:
#divide the journey into two parts
#part 1 is from 0 to 16 included
#part 2 is from 17 to 42 included
# oneVesselLastDataOne = oneVesselLastData.iloc[0:16,:]
# oneVesselLastDataOne = oneVesselLastData.iloc[:,:]
# oneVesselLastDataTwo = oneVesselLastData.iloc[338:350,:]
# oneVesselLastDataOne = oneVesselLastData.iloc[0:15,:]
oneVesselLastDataOne = oneVesselLastData.iloc[:,:]

In [ ]:
print(oneVesselLastDataOne)

In [ ]:
print(oneVesselLastDataTwo)

In [ ]:
oneVessel = "../Data/M120_50_M119_00_33_90_34_44/MMSIContainer/100.csv"
oneVesselLastDataOne,_ = aISDM.load_data_from_csv(oneVessel)

In [ ]:
print(oneVesselLastDataOne)

In [ ]:
fix,ax = plt.subplots()
oneVesselLastDataOneLon = oneVesselLastDataOne["LON"]
oneVesselLastDataOneLat = oneVesselLastDataOne["LAT"]
ax.set_xlim(lonMin, lonMax)
ax.set_ylim(latMin, latMax)
ax.plot(oneVesselLastDataOneLon,oneVesselLastDataOneLat)

In [ ]:
fix,ax = plt.subplots()
oneVesselLastDataTwoLon = oneVesselLastDataTwo["LON"]
oneVesselLastDataTwoLat = oneVesselLastDataTwo["LAT"]
ax.set_xlim(lonMin, lonMax)
ax.set_ylim(latMin, latMax)
ax.plot(oneVesselLastDataTwoLon,oneVesselLastDataTwoLat)

In [ ]:
def generate_vessel_prev_cur_status_(prevState,currState):
    vesselState = np.zeros((prevState.shape[0]*currState.shape[0]))
    stateCounter = 0
    for i in range(prevState.shape[0]):
        for j in range(currState.shape[0]):
            vesselState[stateCounter] = prevState[i]*currState[j]
            stateCounter = stateCounter + 1
    return vesselState

In [ ]:
# from scipy.io import savemat
# from scipy import sparse
# vesselPrevState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[0],oneVesselLastDataOneLat.iloc[0])
# vesselCurrState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[1],oneVesselLastDataOneLat.iloc[1])
# savemat('VesselPrev', {'M':vesselPrevState})
# savemat('VesselCurr', {'M':vesselCurrState})
# savemat('TrPM', {'M':tpMatrix})

In [ ]:
rowSum = np.sum(tpMatrix, axis=1)
rowSumZeroIdx = np.argwhere(rowSum == 0)
rowSumZeroIdx = rowSumZeroIdx.flatten()

In [ ]:
vesselPrevState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[0],oneVesselLastDataOneLat.iloc[0])
vesselCurrState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[1],oneVesselLastDataOneLat.iloc[1])
vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
onePredict = []
for i in range(13):
    vesselProbState = generate_path(vesselState)
    print(i)
    print(np.sum(vesselProbState))
    onePredict.append(vesselProbState.copy())
    vesselPrevState = vesselCurrState.copy()
    vesselCurrState = vesselProbState.copy()
    vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
    #make probabilities zero
    vesselState[rowSumZeroIdx] = 0
    vesselState = vesselState/np.sum(vesselState)
    print(np.sum(vesselState))

In [ ]:
# destDispDir = "../Data/M120_50_M119_00_33_90_34_44/Display/Traj1/"
# for prediction in range(len(onePredict)):
#     opFile = destDispDir + str(prediction) +'.npy'
#     np.save(opFile,onePredict[prediction])
    
# aISDM.save_data_to_csv(oneVesselLastDataOne,(destDispDir+'/True.csv'))

# destDispDir = "../Data/M120_50_M119_00_33_90_34_44/Display/Traj2/"
# for prediction in range(len(onePredict)):
#     opFile = destDispDir + str(prediction) +'.npy'
#     np.save(opFile,onePredict[prediction])
    
# aISDM.save_data_to_csv(oneVesselLastDataOne,(destDispDir+'/True.csv'))

# destDispDir = "../Data/M120_50_M119_00_33_90_34_44/Display/Traj3/"
# for prediction in range(len(onePredict)):
#     opFile = destDispDir + str(prediction) +'.npy'
#     np.save(opFile,onePredict[prediction])
    
# aISDM.save_data_to_csv(oneVesselLastDataOne,(destDispDir+'/True.csv'))

destDispDir = "../Data/M120_50_M119_00_33_90_34_44/Display/Traj4/"
for prediction in range(len(onePredict)):
    opFile = destDispDir + str(prediction) +'.npy'
    np.save(opFile,onePredict[prediction])
    
aISDM.save_data_to_csv(oneVesselLastDataOne,(destDispDir+'/True.csv'))

In [ ]:
boundaryPad = 0.4
horizontalAxisPad, verticalAxisPad, _ = hMUtil.generate_grid((lonMin - boundaryPad) \
                                                             , (lonMax + boundaryPad) \
                                                             , (latMin - boundaryPad) \
                                                             , (latMax + boundaryPad) \
                                                             , increStep \
                                                             , incrRes \
                                                            )
print(horizontalAxisPad.shape)
print(verticalAxisPad.shape)
print(horizontalAxis.shape)
print(verticalAxis.shape)
horzontalPad = (horizontalAxisPad.shape[0] - horizontalAxis.shape[0]) // 2
print(horzontalPad)
verticalPad = (verticalAxisPad.shape[0] - verticalAxis.shape[0]) // 2
print(verticalPad)

In [ ]:
#function which will convert the state vector into formate whih
#can be displayed on the map
def formate_position_data_for_visual(vesselCurrPos):
#     print(vesselCurrPos.shape)    
    #get array of 0s, which needs to be padded from left and right
    #dimension of that would be
    
    horizontalPadNP = np.zeros((verticalAxis.shape[0],horzontalPad))
#     print(horizontalPadNP.shape)
    ret = np.reshape(vesselCurrPos,(verticalAxis.shape[0],horizontalAxis.shape[0]))
    ret = np.flip(ret,axis=0)

    vesselCurrPosPad = np.hstack((horizontalPadNP,ret,horizontalPadNP))
#     print(vesselCurrPosPad.shape)

    verticalPadNP = np.zeros((verticalPad,vesselCurrPosPad.shape[1]))
#     print(verticalPadNP.shape)
    
    vesselCurrPosPad = np.vstack((verticalPadNP,vesselCurrPosPad,verticalPadNP))
#     print(vesselCurrPosPad.shape)
    return vesselCurrPosPad

In [ ]:
vesselCurrPos = get_vessel_init_status(oneVesselLastDataOneLon.iloc[i+2],oneVesselLastDataOneLat.iloc[i+2])

In [ ]:
vesselCurrPosImg = formate_position_data_for_visual(vesselCurrPos)

In [ ]:
unrolledValue = np.where(vesselCurrPosImg > 0)
print(unrolledValue[0][0])
print(unrolledValue[1][0])

In [ ]:
bGImg = plt.imread("ROIResize.png")
print(bGImg.shape)

In [ ]:
print(vesselCurrPosImg.shape)

In [ ]:
def draw_vessels_on_map(x,y):
    ret = np.zeros((bGImg.shape[0], bGImg.shape[1]))
    #get range of pixels in that row to make plus
    rowPixelMin = y - 3
    if(rowPixelMin < 0):
        rowPixelMin = 0
        
    
    rowPixelMax = y + 3 + 1
    #if more than number of rows
    if(rowPixelMax >= bGImg.shape[0]):
        rowPixelMax = bGImg.shape[0]
        
    colPixelMin = x - 3
    if(colPixelMin < 0):
        colPixelMin = 0
        
    colPixelMax = x + 3 + 1
    if(colPixelMax >= bGImg.shape[1]):
        colPixelMax = bGImg.shape[1]
        
    print(rowPixelMin)
    print(rowPixelMax)
    ret[rowPixelMin:rowPixelMax,x] = 1
    ret[y,colPixelMin:colPixelMax] = 1
    
#     ret[rowPixelMin:rowPixelMax,colPixelMin:colPixelMax] = 1
#     return np.flip(ret,axis=0)
    return ret

In [ ]:
tempImg = draw_vessels_on_map(unrolledValue[1][0],unrolledValue[0][0])

In [ ]:
# vesselPrevState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[0],oneVesselLastDataOneLat.iloc[0])
# vesselCurrState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[1],oneVesselLastDataOneLat.iloc[1])
# vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
# useAspect = 0
# aspectRatio = 16.0/9.0
# figWidth = 10
# figHeight = 8

# fig = plt.figure(figsize=(figWidth,figHeight))
fig = plt.figure(figsize=(12,8))
# ax1 = fig.add_axes([0.1, 0.15, 0.49, 0.30])
# ax2 = fig.add_axes([0.1, 0.55, 0.49, 0.30])
# ax4 = fig.add_axes([0.8, 0.55, 0.01, 0.30])

ax1 = fig.add_axes([0.1, 0.1, 0.39, 0.70])
ax2 = fig.add_axes([0.6, 0.1, 0.39, 0.70])
# ax1 = fig.add_axes([0.1, 0.1, 0.45, 0.50])

ax1.imshow(tempImg,cmap=plt.get_cmap('YlGnBu'),extent=[lonMin-boundaryPad \
                                                          ,lonMax+boundaryPad \
                                                          ,latMin-boundaryPad \
                                                          ,latMax+boundaryPad \
                                                         ])

ax1.imshow(bGImg,extent=[lonMin-boundaryPad \
                        ,lonMax+boundaryPad \
                        ,latMin-boundaryPad \
                        ,latMax+boundaryPad \
                        ], alpha = 0.2)

# ax1.plot(oneVesselLastDataOneLon.iloc[i+2],oneVesselLastDataOneLat.iloc[i+2])

ax2.imshow(vesselCurrPosImg,cmap=plt.get_cmap('hot'),extent=[lonMin-boundaryPad \
                                                          ,lonMax+boundaryPad \
                                                          ,latMin-boundaryPad \
                                                          ,latMax+boundaryPad \
                                                         ])

ax1.set_xticks([lonMin-boundaryPad,lonMax+boundaryPad])
ax1.set_yticks([latMin-boundaryPad,latMax+boundaryPad])

ax2.set_xticks([lonMin-boundaryPad,lonMax+boundaryPad])
ax2.set_yticks([latMin-boundaryPad,latMax+boundaryPad])
    
# ax2 = fig.add_axes([0.5, 0.1, 0.49, 0.30])
# ax4 = fig.add_axes([0.9, 0.1, 0.01, 0.30])

In [ ]:
vesselPrevState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[0],oneVesselLastDataOneLat.iloc[0])
vesselCurrState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[1],oneVesselLastDataOneLat.iloc[1])
vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
useAspect = 0
aspectRatio = 16.0/9.0
figWidth = 10
figHeight = 8

fig = plt.figure(figsize=(figWidth,figHeight))
ax1 = fig.add_axes([0.1, 0.15, 0.49, 0.30])
ax2 = fig.add_axes([0.1, 0.55, 0.49, 0.30])
ax4 = fig.add_axes([0.8, 0.55, 0.01, 0.30])

if useAspect == 1:
    figHeight = figWidth/aspectRatio

def animate(i):
    global vesselPrevState, vesselCurrState, vesselState
    vesselCurrPos = get_vessel_init_status(oneVesselLastDataOneLon.iloc[i+2],oneVesselLastDataOneLat.iloc[i+2])
    vesselCurrPos = np.reshape(vesselCurrPos,(verticalAxis.shape[0],horizontalAxis.shape[0]))
    
    vesselCurrPos = np.flip(vesselCurrPos,axis=0)

    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Lattitude')
    titleStr = 'Vessel Position %d'%((i))
    ax1.set_title(titleStr)
    ax1.set_xticks([lonMin,lonMax])
    ax1.set_yticks([latMin,latMax])

    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Lattitude')
    ax2.set_title('Transition Probability Projection')
    ax2.set_xticks([lonMin,lonMax])
    ax2.set_yticks([latMin,latMax])

    ax1.imshow(vesselCurrPos,cmap=plt.get_cmap('hot'),extent=[lonMin,lonMax,latMin,latMax])
    #sequence of vessel states
#     vesselProbState = generate_path(vesselState)
#     #copy the file
#     vesselPrevState = vesselCurrState.copy()
#     vesselCurrState = vesselProbState.copy()
#     vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
    
    vesselProbState = onePredict[i].copy()
    vesselProbState = np.reshape(vesselProbState,(verticalAxis.shape[0],horizontalAxis.shape[0]))
    vesselProbState = np.flip(vesselProbState,axis=0)
    iMShowHandler = ax2.imshow(vesselProbState,cmap=plt.get_cmap('YlGnBu'),extent=[lonMin,lonMax,latMin,latMax], vmax = 0.1)
    
    fig.colorbar(iMShowHandler,ax4)
    
anim = animation.FuncAnimation(fig, animate,frames=13, interval=1000)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=1, metadata=dict(artist='Me'), bitrate=1800)

In [ ]:
# anim.save('TPSeqOneSecOrder.mp4', writer=writer)
# anim.save('TPSeqTwoSecOrder.mp4', writer=writer)
# anim.save('FishingVessel.mp4', writer=writer)
anim.save('NewRegion3.mp4', writer=writer)

In [ ]:
vesselPrevState = get_vessel_init_status(oneVesselLastDataTwoLon.iloc[0],oneVesselLastDataTwoLat.iloc[0])
vesselCurrState = get_vessel_init_status(oneVesselLastDataTwoLon.iloc[1],oneVesselLastDataTwoLat.iloc[1])
vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
twoPredict = []
for i in range(20):
    vesselProbState = generate_path(vesselState)
    print(i)
    print(np.sum(vesselProbState))
    twoPredict.append(vesselProbState.copy())
    vesselPrevState = vesselCurrState.copy()
    vesselCurrState = vesselProbState.copy()
    vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
    
    vesselState[rowSumZeroIdx] = 0
    vesselState = vesselState/np.sum(vesselState)

In [ ]:
vesselPrevState = get_vessel_init_status(oneVesselLastDataTwoLon.iloc[0],oneVesselLastDataTwoLat.iloc[0])
vesselCurrState = get_vessel_init_status(oneVesselLastDataTwoLon.iloc[1],oneVesselLastDataTwoLat.iloc[1])
vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)

useAspect = 0
aspectRatio = 16.0/9.0
figWidth = 10
figHeight = 8

fig = plt.figure(figsize=(figWidth,figHeight))
ax1 = fig.add_axes([0.1, 0.15, 0.49, 0.30])
ax2 = fig.add_axes([0.1, 0.55, 0.49, 0.30])
ax4 = fig.add_axes([0.8, 0.55, 0.01, 0.30])

if useAspect == 1:
    figHeight = figWidth/aspectRatio

def animate(i):
    global vesselPrevState, vesselCurrState, vesselState
    vesselCurrPos = get_vessel_init_status(oneVesselLastDataTwoLon.iloc[i+2],oneVesselLastDataTwoLat.iloc[i+2])
    vesselCurrPos = np.reshape(vesselCurrPos,(verticalAxis.shape[0],horizontalAxis.shape[0]))
    
    vesselCurrPos = np.flip(vesselCurrPos,axis=0)

    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Lattitude')
    titleStr = 'Vessel Position %d'%((i))
    ax1.set_title(titleStr)
    ax1.set_xticks([lonMin,lonMax])
    ax1.set_yticks([latMin,latMax])

    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Lattitude')
    ax2.set_title('Transition Probability Projection')
    ax2.set_xticks([lonMin,lonMax])
    ax2.set_yticks([latMin,latMax])

    ax1.imshow(vesselCurrPos,cmap=plt.get_cmap('hot'),extent=[lonMin,lonMax,latMin,latMax])
    #sequence of vessel states
    
    vesselProbState = twoPredict[i].copy()
    vesselProbState = np.reshape(vesselProbState,(verticalAxis.shape[0],horizontalAxis.shape[0]))
    vesselProbState = np.flip(vesselProbState,axis=0)
    
    iMShowHandler = ax2.imshow(vesselProbState,cmap=plt.get_cmap('YlGnBu'),extent=[lonMin,lonMax,latMin,latMax], vmax = 0.1)
    
    fig.colorbar(iMShowHandler,ax4)
    
anim = animation.FuncAnimation(fig, animate,frames=20, interval=1000)


In [ ]:
HTML(anim.to_html5_video())

In [ ]:
anim.save('TPSeqTwoSecOrder.mp4', writer=writer)